# Explore here

In [2]:
import pandas as pd
from pathlib import Path
import zipfile, os
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

import warnings
import joblib
warnings.filterwarnings('ignore')

In [3]:
#Ubicamos modelo
#Indicamos cuál es la carpeta en directorio (src)
BASE_DIR = Path().resolve()

#Indicamos la ruta del modelo
MODEL_PATH = BASE_DIR.parent / "models" / "xgb_model.pkl"

print(MODEL_PATH)
print(MODEL_PATH.exists())  #Comprobamos que el modelo existe = true

/workspaces/equipoEnergias_XGB_con_Streamlit/models/xgb_model.pkl
True


In [4]:
#Cargamos modelo
MODEL_PATH = "../models/xgb_model.pkl"
model = joblib.load(MODEL_PATH)
print("El modelo se ha cargado correctamente ✅")

El modelo se ha cargado correctamente ✅


In [5]:
#Sacamos las categorias usadas del modelo
print(model.feature_names_in_)

['demanda_lag_1' 'demanda_lag_24' 'demanda_lag_168' 'media_movil_24h'
 'hora' 'mes' 'es_finde' 'dia_semana' 'Madrid_temperature_2m'
 'Valencia_temperature_2m' 'Pais_Vasco_temperature_2m'
 'Cataluna_temperature_2m' 'Andalucia_temperature_2m']


In [7]:
#Probamos a predecir
X_test = pd.DataFrame([{
    "demanda_lag_1": 28000,
    "demanda_lag_24": 27500,
    "demanda_lag_168": 26000,
    "media_movil_24h": 27000,
    "hora": 18,
    "mes": 1,
    "es_finde": 0,
    "dia_semana": 2,
    "Madrid_temperature_2m": 30.5,
    "Valencia_temperature_2m": 29.0,
    "Pais_Vasco_temperature_2m": 22.0,
    "Cataluna_temperature_2m": 28.0,
    "Andalucia_temperature_2m": 33.0
}])

X_test = X_test[model.feature_names_in_]
pred = model.predict(X_test)
print("Predicción de demanda eléctrica (MW):", pred[0])

Predicción de demanda eléctrica (MW): 28347.047


In [8]:
#para el error de DATABASE_URL
def db_connect():
    db_url = os.getenv("DATABASE_URL")

    if not db_url:
        return None  # ← evita el crash

    engine = create_engine(db_url)
    return engine

In [10]:
print("Columnas que espera el modelo:")
print(model.feature_names_in_)

Columnas que espera el modelo:
['demanda_lag_1' 'demanda_lag_24' 'demanda_lag_168' 'media_movil_24h'
 'hora' 'mes' 'es_finde' 'dia_semana' 'Madrid_temperature_2m'
 'Valencia_temperature_2m' 'Pais_Vasco_temperature_2m'
 'Cataluna_temperature_2m' 'Andalucia_temperature_2m']


# -----------------------------
# PARTE 1: Inputs de demanda
# -----------------------------
st.title("Predicción de Demanda Eléctrica ⚡")
st.subheader("Introduce los valores")

def float_input_safe(label, ejemplo=27000):
    # Contenedor horizontal
    col_input, col_ej = st.columns([0.2, 0.4])

    with col_input:
        val_str = st.text_input(
            f"{label} (MW)", 
            value="", 
            max_chars=10,
            key=label,
            help=f"Ej. {ejemplo}"  # ahora la guía está en el tooltip de ayuda
        )

        # Conversión segura a float
        try:
            val_clean = float(val_str.replace(".", "").replace(",", "")) if val_str else ejemplo
        except:
            val_clean = ejemplo

    with col_ej:
        # Columna vacía, ya no necesitamos la leyenda fuera de la caja
        st.write("")

    return val_clean

demanda_lag_1 = float_input_safe("Demanda hace 1 hora")
demanda_lag_24 = float_input_safe("Demanda hace 24 horas")
demanda_lag_168 = float_input_safe("Demanda hace 168 horas")
media_movil_24h = float_input_safe("Media móvil 24h")


# -----------------------------
# Slider interactivo de hora con color fijo azul cobalto
# -----------------------------
col1, col2 = st.columns([2,1])

with col1:
    # Slider de hora
    hora_real = st.slider(
        "Hora del día",
        min_value=0,
        max_value=23,
        value=18,  # valor por defecto = 6 PM
        step=1
    )

icono = "☀️" if 6 <= hora_real <= 18 else "🌙"

st.markdown(f"""
<style>
div[data-baseweb="slider"] input[type="range"] {{
    accent-color: #0047AB;
}}
</style>
""", unsafe_allow_html=True)

st.markdown(f"<div style='margin-top:5px; margin-bottom:10px; color:#0047AB; font-weight:bold;'>Hora seleccionada: {hora_real}h {icono}</div>", unsafe_allow_html=True)


# -----------------------------
# Slider para el día de la semana
# -----------------------------

dias_semana_nombres = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

with col1:
    dia_semana = st.slider(
        "Día de la semana",
        min_value=1,
        max_value=7,
        value=3,  # Por defecto Miércoles
        step=1
    )

dia_nombre = dias_semana_nombres[dia_semana - 1]

es_finde_num = 1 if dia_semana in [6, 7] else 0
es_finde_texto = "Sí" if es_finde_num == 1 else "No"

st.markdown(f"<div style='margin-top:5px; margin-bottom:5px; font-weight:bold;'>Día seleccionado: {dia_nombre}</div>", unsafe_allow_html=True)
st.markdown(f"<div style='margin-bottom:10px;'>Es fin de semana: {es_finde_texto}</div>", unsafe_allow_html=True)


# -----------------------------
# MES + ESTACIÓN DEL AÑO
# -----------------------------

meses = {
    "Enero": 1,
    "Febrero": 2,
    "Marzo": 3,
    "Abril": 4,
    "Mayo": 5,
    "Junio": 6,
    "Julio": 7,
    "Agosto": 8,
    "Septiembre": 9,
    "Octubre": 10,
    "Noviembre": 11,
    "Diciembre": 12
}

col1, col2 = st.columns([0.2,0.4])
with col1:
    mes_nombre = st.selectbox("Mes", list(meses.keys()))
    mes = meses[mes_nombre]

if mes in [12, 1, 2]:
    estacion = "❄️ Invierno"
elif mes in [3, 4, 5]:
    estacion = "🌱 Primavera"
elif mes in [6, 7, 8]:
    estacion = "☀️ Verano"
else:
    estacion = "🍂 Otoño"

st.markdown(f"<div style='margin-top:5px; margin-bottom:15px; font-weight:bold;'>Estación del año: {estacion}</div>", unsafe_allow_html=True)


# -----------------------------
# TEMPERATURA SEGÚN REGIÓN
# -----------------------------
st.markdown("### 🌡️ Temperaturas por región")

temp_valores = list(range(-15, 49))

col1, col2, col3 = st.columns(3)

with col1:
    temp_mad = st.selectbox("Región Central (ºC)", temp_valores, index=temp_valores.index(30))
    temp_val = st.selectbox("Región Sureste (ºC)", temp_valores, index=temp_valores.index(29))

with col2:
    temp_pv = st.selectbox("Región Norte (ºC)", temp_valores, index=temp_valores.index(22))
    temp_cat = st.selectbox("Región Noreste (ºC)", temp_valores, index=temp_valores.index(28))

with col3:
    temp_and = st.selectbox("Región Sur (ºC)", temp_valores, index=temp_valores.index(33))


# -----------------------------
# DataFrame para el modelo
# -----------------------------
X_input = pd.DataFrame([{
    "demanda_lag_1": demanda_lag_1,
    "demanda_lag_24": demanda_lag_24,
    "demanda_lag_168": demanda_lag_168,
    "media_movil_24h": media_movil_24h,
    "hora": hora_real,
    "mes": mes,
    "es_finde": es_finde_num,
    "dia_semana": dia_semana,
    "Madrid_temperature_2m": temp_mad,
    "Valencia_temperature_2m": temp_val,
    "Pais_Vasco_temperature_2m": temp_pv,
    "Cataluna_temperature_2m": temp_cat,
    "Andalucia_temperature_2m": temp_and
}])

# -----------------------------
# Alineación con columnas del modelo
# -----------------------------
for col in model.feature_names_in_:
    if col not in X_input.columns:
        X_input[col] = 0.0
X_input = X_input[model.feature_names_in_]

# -----------------------------
# Predicción
# -----------------------------
if st.button("Calcular"):
    pred = model.predict(X_input)[0]
    st.success(f"📈 La predicción de demanda real es de **{pred:,.0f} MW**")

# -----------------------------
# SECCIÓN EDA
# -----------------------------
if seccion == "EDA":
    st.title("📊 Análisis Exploratorio de Datos (EDA)")
    st.info("inserte aquí verborrea y grafiquitos.")
